In [1]:
#Libraries
import numpy as np # to perform algebric calculations
import pandas as pd # data processing-input as pandas dataframe
import os
import gc # To clean memory
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
#import seaborn as sns
#sns.set_style('whitegrid')
#import matplotlib.pyplot as plt  #for plotting purpose
#from scipy.sparse import lil_matrix
#from sklearn.ensemble import RandomForestClassifier

In [2]:
###################################################################
#                       READ DATA                                 #
###################################################################
clicks_train = pd.read_csv("../data/clicks_train.csv")
clicks_train.fillna(0,inplace=True)

promoted_content = pd.read_csv("../data/promoted_content.csv")
promoted_content.fillna(0, inplace=True)
X_test=pd.read_csv("../data/clicks_test.csv")
print("file_read done")
####################################################################

file_read done


In [3]:
k=int(clicks_train.clicked.mean())
M=k
df_mrg=clicks_train.merge(promoted_content,on='ad_id' ,how='left')

In [ ]:
#Analysis of ads_prob in clicks_train.csv
train_prob=clicks_train.groupby(['ad_id']).clicked.agg(['count' ,'sum']).reset_index()
k=int(clicks_train.clicked.mean())
M=k
train_prob['prob_Score']=((train_prob['sum']+18*k)/(18+train_prob['count']))
train_prob=train_prob.drop(['count','sum'],1)
#print(train_prob)

In [ ]:
#Add the calculated scores to promoted_content based on 'ad_id'
promoted_content=promoted_content.merge(train_prob,on='ad_id' ,how='left')
promoted_content.fillna(0, inplace=True)
del train_prob
print("prob_score cal done")

In [13]:
events = pd.read_csv("../data/events.csv")
#events.fillna(0, inplace=True)
events=events[['display_id','platform']]
#ad_platform_score from events file calculation
events.platform = events.platform.astype(str)
print(events)
#events.platform = events.platform.replace('\\N','2',inplace=True)
print(events.platform.unique())

          display_id platform
0                  1        3
1                  2        2
2                  3        2
3                  4        2
4                  5        2
5                  6        3
6                  7        2
7                  8        2
8                  9        1
9                 10        1
10                11        3
11                12        1
12                13        1
13                14        3
14                15        2
15                16        1
16                17        2
17                18        2
18                19        3
19                20        2
20                21        2
21                22        2
22                23        2
23                24        2
24                25        2
25                26        2
26                27        2
27                28        2
28                29        2
29                30        2
...              ...      ...
23120096    23120097        3
23120097  

In [14]:

df_mrg1=clicks_train.merge(events,on='display_id' ,how='left')
df_mrg1['ad_id_pl']= df_mrg1['ad_id'].map(str) + "_" + df_mrg1.platform
df_events=df_mrg1.groupby('ad_id_pl').clicked.agg(['count' ,'sum']).reset_index()
df_events['ad_pl_Score']= (df_events['sum']+18*k ) / (18 + df_events['count'])


In [15]:
df_events=df_mrg1.merge(df_events,on='ad_id_pl' ,how='left')
df_events=df_events.drop(['platform','count','sum'],1)
print("clicks_train+events done")
X_test=X_test.merge(events,on='display_id' ,how='left')
X_test['ad_id_pl']= X_test['ad_id'].map(str) + "_" + X_test.platform
print("clicks_test+events done")
del events
del df_mrg1,clicks_train

clicks_train+events done
clicks_test+events done


In [16]:
#join click_trains with  promoted_content on ad_id and calculate campaigner_score
df_cmpg= df_mrg.groupby('campaign_id').clicked.agg(['count' ,'sum']).reset_index()
df_cmpg['cmpg_Score']= (df_cmpg['sum'] +18*M) / (18 + df_cmpg['count'])
df_cmpg=pd.DataFrame(df_cmpg).sort('cmpg_Score')
df_cmpg=df_cmpg.drop('count',1)
df_cmpg=df_cmpg.drop('sum',1)

In [17]:
#join click_trains with  promoted_content on ad_id and calculate advertiser_score
df_adv= df_mrg.groupby('advertiser_id').clicked.agg(['count' ,'sum']).reset_index()
df_adv['adv_Score']= (df_adv['sum'] + M*18) / (18 + df_adv['count'])
df_adv=df_adv.drop('count',1)
df_adv=df_adv.drop('sum',1)

In [18]:
#add the calculated campaigner_score and advertiser_score to promoted_content
promoted_content=promoted_content.merge(df_adv,on='advertiser_id',how='left').merge(df_cmpg,on='campaign_id',how='left')
promoted_content=promoted_content.drop('campaign_id',1)
promoted_content=promoted_content.drop('advertiser_id',1)
del df_adv,df_cmpg,df_mrg

print("campg_score and adv_score done")


campg_score and adv_score done


In [19]:
print(promoted_content)

         ad_id  document_id  adv_Score  cmpg_Score
0            1         6614   0.202470    0.240752
1            2       471467   0.202470    0.125249
2            3         7692   0.202470    0.113260
3            4       471471   0.202470    0.125249
4            5       471472   0.202470    0.125249
5            6        12736   0.202470    0.240752
6            7        12808   0.202470    0.240752
7            8       471477   0.202470    0.125249
8            9        13379   0.202470    0.240752
9           10        13885   0.202470    0.240752
10          11        14230   0.202470    0.240752
11          12       446701   0.113761    0.067039
12          13       471499   0.113761    0.067039
13          14       471500   0.113761    0.067039
14          15       471501   0.113761    0.067039
15          16       471514   0.113761    0.053763
16          17       471517   0.113761    0.067039
17          18       471518   0.113761    0.067039
18          19       471519   0

In [20]:
#create final training_data file -merge promoted_content and df_events
gc.collect()
X=df_events.merge(promoted_content,how='left',on='ad_id')
promoted_content=promoted_content.drop('document_id',1)
print("X (join of pl_score and promoted_content) creation done")
df=df_events.drop(['ad_id','display_id','clicked'],1)
del df_events
df=df.drop_duplicates()

X (join of pl_score and promoted_content) creation done


In [21]:
Y=np.array(X['clicked'])
X= X.drop(['ad_id','display_id','clicked','document_id','ad_id_pl'],1)
X=np.array (X)
print("X array created")

X array created


In [22]:
print(X)
print(Y)
print(df)

[[ 0.19629527  0.2225146   0.16701992]
 [ 0.1689008   0.170883    0.15096548]
 [ 0.29403468  0.31435508  0.31435508]
 ..., 
 [ 0.37459283  0.21405144  0.35585415]
 [ 0.24398178  0.18644197  0.1679962 ]
 [ 0.17469011  0.14241466  0.14241466]]
[0 0 1 ..., 0 0 0]
          ad_id_pl  ad_pl_Score
0          42337_3     0.196295
1         139684_3     0.168901
2         144739_3     0.294035
3         156824_3     0.039135
4         279295_3     0.264000
5         296965_3     0.000000
6         125211_2     0.150410
7         156535_2     0.000000
8         169564_2     0.094862
9         308455_2     0.352018
10         71547_2     0.080171
11         95814_2     0.235476
12        152141_2     0.163235
13        183846_2     0.202232
14        228657_2     0.154098
15        250082_2     0.038334
16        149930_2     0.146012
17        153623_2     0.233444
18        184709_2     0.069848
19        186849_2     0.149480
20        233730_2     0.144254
21         54606_2     0.208232
22 

In [23]:
X_test=X_test.merge(df,on='ad_id_pl' ,how='left')
del df
gc.collect()
X_test=X_test.merge(promoted_content,how='left',on='ad_id')
del promoted_content
X_test= X_test.drop(['display_id','ad_id','platform','ad_id_pl'],1)
X_test.fillna(value=0,inplace=True)
print("X_test created")

X_test created


In [ ]:
from sklearn.svm import SVC
SVC(C=1.0, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=2, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf=SVC()
clf.fit(X,Y)
print(clf.score(X,Y))

In [24]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(X, Y)
print(accuracy_score(Y, gnb_clf.predict(X)))
print(gnb_clf.score(X,Y))

0.777150249632
0.777150249632


In [25]:
#Applying Model
gc.collect()
reg=SGDClassifier(loss= 'log', penalty= 'l2')
reg.fit(X,Y)
print(accuracy_score(Y, reg.predict(X)))
print(reg.score(X,Y))
output= reg.decision_function(X_test)
#plt.hist(output)
#plt.show()
print("model fit done")

0.809795068221
0.809795068221
model fit done


In [26]:
#Create submission File
output=pd.read_csv('../data/clicks_test.csv').merge(pd.DataFrame(output,columns=['prediction']), left_index=True,right_index=True)
output.sort_values(['display_id','prediction'], inplace=True, ascending=False)
#.isin(16874594,16874595,16874596,16874597,16874598)].sort('prob_Score'))
subm = output.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()
print(subm[subm["display_id"]==16874594])
subm.to_csv('../output/submission_file_11dec_final.csv', index=False)
print("output stored")

   display_id                                     ad_id
0    16874594  170392 172888 162754 66758 150083 180797
output stored
